In [72]:
!pip install flask
!pip install pyserial
!jupyter nbconvert --to script functions.ipynb
!pip install nbimporter

[NbConvertApp] Converting notebook functions.ipynb to script
[NbConvertApp] Writing 4198 bytes to functions.py


In [12]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/')
def index():
    return 'Welcome to my capstone server'

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [10/May/2024 12:01:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/May/2024 12:01:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2024 12:01:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/May/2024 12:01:06] "GET /favicon.ico HTTP/1.1" 404 -


In [70]:
from flask import Flask
import serial
import threading
import subprocess
import nbimporter

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM3', 9600)

def read_from_port(ser):
    while True:
        reading = ser.readline().decode().strip()  # 끝의 줄바꿈 제거
        print("Received from Arduino:", reading)
        if reading == "Take Picture":
            take_picture()
        elif reading == "Next":
            next_text()
        elif reading == "Previous":
            previous_text()
        else:
            print("Received from Arduino:", reading)

def take_picture():
    print("Taking picture...")
    # 'main.ipynb' 노트북을 실행하는 명령어
    command = ['jupyter', 'notebook', 'main.ipynb']
    # subprocess를 이용하여 명령어 실행
    subprocess.run(command)
    print("main.ipynb 실행됨...")
    
def next_text():
    print("Showing next text...")

def previous_text():
    print("Showing previous text...")

@app.route('/')
def index():
    return "Serial Communication with Arduino"

thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.18.234:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.18.234:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit


Received from Arduino: Next
Showing next text...
Received from Arduino: Previous
Showing previous text...
